# 🚀 DeepSeek-R1 Model with Azure AI Inference 🧠

**DeepSeek-R1** es un modelo de razonamiento de vanguardia que combina aprendizaje por refuerzo y ajuste fino supervisado, destacándose en tareas de razonamiento complejas con 37B parámetros activos y una ventana de contexto de 128K.

En este cuaderno de Jupyter aprenderás a:
1. **Initialize** el ChatCompletionsClient para endpoints sin servidor de Azure
2. **Chat** con DeepSeek-R1 utilizando la extracción de razonamiento
3. **Implement** un ejemplo de planificación de viajes con razonamiento paso a paso
4. **Leverage** la ventana de contexto de 128K para escenarios complejos

## ¿Por qué DeepSeek-R1?
- **Advanced Reasoning**: Se especializa en la resolución de problemas mediante cadenas de pensamiento
- **Massive Context**: Ventana de 128K tokens para análisis detallado
- **Efficient Architecture**: 37B parámetros activos de un total de 671B
- **Safety Integrated**: Capacidades integradas de filtrado de contenido


## 1. Setup

Importar la libreria
- **azure-ai-projects**: para el `AIProjectClient`.
- **azure-ai-inference**: para llamar los modelos
- **azure-identity**: para `DefaultAzureCredential`.

Proporciona un archivo `.env` con:
```bash
PROJECT_CONNECTION_STRING=<your-conn-string>
```

In [20]:
import os
from dotenv import load_dotenv
from pathlib import Path
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.inference.models import SystemMessage, UserMessage, AssistantMessage
import re

from pathlib import Path

# Load environment variables
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir / '../.env', override=True)

conn_string = os.getenv("PROJECT_CONNECTION_STRING")
model_name = "DeepSeek-R1"

try:
    project_client = AIProjectClient.from_connection_string(
        credential=DefaultAzureCredential(),
        conn_str=conn_string,
    )
    print("✅ AIProjectClient created successfully!")
except Exception as e:
    print("❌ Error creating AIProjectClient:", e)

✅ AIProjectClient created successfully!


## 2. Intelligent Travel Planning ✈️

Demonstrate DeepSeek-R1's reasoning capabilities for trip planning:

In [21]:
def plan_trip_with_reasoning(query, show_thinking=False):
    """Get travel recommendations with reasoning extraction"""
    messages = [
        SystemMessage(content="Eres un experto en viajes. Proporciona planes detallados con justificación."),
        UserMessage(content=f"{query} Incluye gemas ocultas y consideraciones de seguridad.")
    ]

    with project_client.inference.get_chat_completions_client() as chat_client:
        response = chat_client.complete(
            messages=messages,
            model=model_name,
            temperature=0.7,
            max_tokens=1024
        )
    
    content = response.choices[0].message.content
    
    # Extract reasoning if present
    if show_thinking:
        match = re.search(r"<think>(.*?)</think>(.*)", content, re.DOTALL)
        if match:
            return {"thinking": match.group(1).strip(), "answer": match.group(2).strip()}
    return content

# Example usage
query = "Planifica un viaje cultural de 5 días a Kioto en abril"
result = plan_trip_with_reasoning(query, show_thinking=True)

print("🗺️ Query:", query)
if isinstance(result, dict):
    print("\n🧠 Thinking Process:", result["thinking"])
    print("\n📝 Final Answer:", result["answer"])
else:
    print("\n📝 Response:", result)

🗺️ Query: Planifica un viaje cultural de 5 días a Kioto en abril

🧠 Thinking Process: Okay, the user wants a 5-day cultural trip to Kyoto in April, including hidden gems and safety considerations. Let me start by recalling Kyoto's main attractions and the less-known spots. April is cherry blossom season, so hanami spots are a must, but maybe suggest some quieter ones to avoid crowds.

First day: Arrival and central Kyoto. Kiyomizu-dera is a must, but maybe add nearby hidden spots like Sannen-zaka and Ninen-zaka. They’re historic streets but can get crowded. Safety tip: watch out for slippery stone paths, especially if it rains.

Second day: Arashiyama. Bamboo Grove is popular, so suggest going early. Then Okochi Sanso Villa as a hidden gem. Maybe include Gio-ji Temple in the afternoon, which is quieter. Safety in bamboo grove: stay on paths to avoid getting lost.

Third day: Northern Kyoto. Ryoan-ji and Ninna-ji are well-known temples. Add Myoman-ji Temple as a hidden gem. Ohara villag

## 3. Technical Problem Solving 💻

Showcase coding/optimization capabilities:

In [22]:
def solve_technical_problem(problem):
    """Solve complex technical problems with structured reasoning"""
    with project_client.inference.get_chat_completions_client() as chat_client:
        response = chat_client.complete(
            messages=[
            UserMessage(content=f"{problem} Por favor razona paso a paso, y coloca tu respuesta final dentro de \\boxed{{}}.")
            ],
            model=model_name,
            temperature=0.3,
            max_tokens=2048
        )
    
    return response.choices[0].message.content

# Database optimization example
problem = """¿Cómo puedo optimizar una base de datos PostgreSQL que maneja 10k transacciones/segundo?
Considera estrategias de indexación, requisitos de hardware y optimización de consultas."""

print("🔧 Problem:", problem)
print("\n⚙️ Solution:", solve_technical_problem(problem))

🔧 Problem: ¿Cómo puedo optimizar una base de datos PostgreSQL que maneja 10k transacciones/segundo?
Considera estrategias de indexación, requisitos de hardware y optimización de consultas.

⚙️ Solution: <think>
Okay, I need to figure out how to optimize a PostgreSQL database handling 10k transactions per second. That's a high workload, so the solution has to be thorough. Let me start by breaking down the problem into parts: indexing strategies, hardware requirements, and query optimization. 

First, indexing. I know that proper indexing is crucial for performance. But using too many indexes can slow down writes. So maybe I should look at the most frequent queries and see which columns are used in WHERE clauses, JOINs, or ORDER BY. Maybe using B-tree indexes for common lookups. Also, composite indexes if queries filter on multiple columns. But wait, the order of columns in composite indexes matters. They should be ordered from the most specific to the least specific. What about indexes 